<a href="https://colab.research.google.com/github/vladang73/RESTFulAPP/blob/master/CTRL-ft-02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, make sure a GPU is attached. 

In [1]:
!nvidia-smi

Sun May 17 12:32:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!git clone https://github.com/salesforce/ctrl

Cloning into 'ctrl'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 157 (delta 10), reused 11 (delta 5), pack-reused 136
Receiving objects: 100% (157/157), 24.46 MiB | 24.60 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [3]:
%cd ctrl/

/content/ctrl


To allow for generation on GPUs with lower memory, simply check out the `lower_memory` branch. This is still under testing so generations may not be as good. I will update the repo (and possibly merge it into master) as the behavior stabilizes. 

In [4]:
!git checkout lower_memory

Branch 'lower_memory' set up to track remote branch 'lower_memory' from 'origin'.
Switched to a new branch 'lower_memory'


In [5]:
!pip2 install tensorflow-gpu==1.14

     |████████████████████████████████| 377.0MB 42kB/s 
     |████████████████████████████████| 3.2MB 41.0MB/s 
     |████████████████████████████████| 491kB 69.6MB/s 
ERROR: tensorflow 2.1.0 has requirement tensorboard<2.2.0,>=2.1.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 2.1.0 has requirement tensorflow-estimator<2.2.0,>=2.1.0rc0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0


In [6]:
!patch -b /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch

patching file /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py


In [7]:
!pip2 install fastBPE

  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp27-cp27mu-linux_x86_64.whl size=476469 sha256=baf50168011d4d4eb3b52e8f4f99856d6c46d059e430ad1a76d20d7ee0c36664
  Stored in directory: /root/.cache/pip/wheels/f3/0c/9c/fc62058b4d473a5602bcd3d3edfece796f123875379ea82d79
Successfully built fastBPE


In [8]:
!pip2 install gsutil

     |████████████████████████████████| 2.3MB 4.8MB/s 
     |████████████████████████████████| 143kB 23.1MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 2.7MB 51.3MB/s 
  Created wheel for gsutil: filename=gsutil-4.50-cp27-none-any.whl size=3041246 sha256=2ce080f9c935e9dcbcc4198e495338034bdf7178becc99b02ac1e84a109ef11d
  Stored in directory: /root/.cache/pip/wheels/d6/c7/fb/7cab278a21c09dd37a222b39c71bd68bdc884ca42da155b5d0
  Created wheel for gcs-oauth2-boto-plugin: filename=gcs_oauth2_boto_plugin-2.5-cp27-none-any.whl size=19000 sha256=c62e160aa1d55338a24d2c409660a98bf290327cd9e66d2ea6a9510a6c994274
  Stored in directory: /root/.cache/pip/wheels/12/98/2d/f3ef8476709ea17c646a4606a32c0273b5dbb48ef36f4fb33d
  Created wheel for retry-decorator: filename=retry_decorator-1.1.1-py2.py3-none-any.whl size=3639 sha256=3a994171acfbeaa3bb9233cf00d88e66a8a1c71bddadf2c155ec487a9ee6c99b
  Stored in directory: /root/.cache/pip/wheels/a1/70/30/4af82

In [9]:
!gsutil -m cp -r gs://sf-ctrl/seqlen256_v1.ckpt .

Copying gs://sf-ctrl/seqlen256_v1.ckpt/checkpoint...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.meta...
Copying gs://sf-ctrl/seqlen256_v1.ckpt/model.ckpt-413000.index...
| [4/4 files][ 12.7 GiB/ 12.7 GiB] 100% Done  37.8 MiB/s ETA 00:00:00           
Operation completed over 4 objects/12.7 GiB.                                     


Now, let's generate. The script will print the whole prompt after generating each token. You can turn this off and generate only once at the end by also adding the flag `--print_once`. Scroll to the bottom of the next cell to see the final generated output.

There are some gotchas here to be aware of. 

1. Unlike the base version, here, the `model_path` requires the path to the `data` file and not just the ckpt folder. 

2. The first generation is slow because of overhead but the subsequent ones should be fast. 

3. I have added fine-tuning `--temperature 0.5`  

The prompt is `Links https://www.buzzfeednews.com/article/annehelenpetersen/jeremy-renner-app-trolling-career-hurt-locker-avengers`


In [10]:
!python2 generation.py --temperature 0.5 --model seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001

W0517 12:41:09.139540 139669582628736 deprecation_wrapper.py:119] From generation.py:6: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.

W0517 12:41:09.175122 139669582628736 deprecation_wrapper.py:119] From generation.py:43: The name tf.random.set_random_seed is deprecated. Please use tf.compat.v1.random.set_random_seed instead.

246534 unique words
2020-05-17 12:41:09.497953: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-05-17 12:41:09.540777: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-17 12:41:09.541361: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-05-17 1

Now let's make some fine-tuning using 1 sentence. 

Adding `--temperature 0.5`  and  `--penalty 1.4`

The prompt is `Links https://www.usnews.com/news/national-news/articles/2020-05-16/who-releases-brief-on-rare-illness-in-children-possibly-linked-to-the-coronavirus`

In [12]:
!python2 generation.py --temperature 0.5 --penalty 1.4 --model seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001

W0517 13:32:55.764539 140316048000896 deprecation_wrapper.py:119] From generation.py:6: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.

W0517 13:32:55.811980 140316048000896 deprecation_wrapper.py:119] From generation.py:43: The name tf.random.set_random_seed is deprecated. Please use tf.compat.v1.random.set_random_seed instead.

246534 unique words
2020-05-17 13:32:56.115443: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-05-17 13:32:56.133842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-17 13:32:56.134390: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-05-17 1